In [ ]:
import requests
import json
import pprint

with open('city.list_JP.json', mode='r', encoding='utf-8') as f:
    city_list = json.loads(f.read())

city_list

In [ ]:
# config.jsonによけたAPI_keyの取得
with open('config.json', mode='r', encoding='utf-8') as f:
    jsonfile = json.loads(f.read())
API_key = jsonfile['API_key']

In [ ]:
# 明日からの作業：
# https://openweathermap.org/forecast5#name5
# ここを参考にして各地の天気データを取得する
city_name = 'Tokyo'
flg = False
for list in city_list:
    if city_name == list['name']:
        print("ありました")
        flg = True
if flg == False:
    print("ありませんでした")

In [ ]:
# 特定の市町村の緯度経度を入手したい
city_name = 'Nagoya'
flg = False
for city in city_list:
    if city_name == city['name']:
        pprint.pprint(city)
        lat  = city['coord']['lat']
        lon  = city['coord']['lon']
        flg = True
if flg == False:
    print("ありませんでした")

# せっかくだからfoliumでも遊ぶ
# とりあえず地図の中央を可児市にしておく
import folium

m = folium.Map(location=(lat, lon),  zoom_start=5) #lat, lonの順番

In [ ]:
# 複数都市を指定するには配列入りオブジェクトとかにして、そこに都市名・coordを取得。
# そのデータをfolium.Markerに渡していけばよさそう。

folium.Marker(
    location=[35.689499, 139.691711],
    tooltip="Click me!",
    popup="Tokyo",
    icon=folium.Icon(color="red"),
).add_to(m)

folium.Marker(
    location=[35.950001, 137.266663],
    tooltip="Click me!",
    popup="Osaka",
    icon=folium.Icon(color="green"),
).add_to(m)

m

In [ ]:
city_name = 'Wakkanai'
url = f'http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=5&appid={API_key}'
jsondata = requests.get(url).json()
pprint.pprint(jsondata)

In [ ]:
jsondata[0]['local_names']['ja']

In [ ]:
# ...なんでBetter Commentsは一行目は色変えられないんですかね…
#* 取得したい都市名：稚内、札幌、仙台、新潟、東京、金沢、長野、大阪、福山、高知、福岡、鹿児島、那覇
#! 大阪市はid:1853909。1853908の方は岐阜県下呂市にある「小坂」という場所の模様
selected_cities_name = ['Wakkanai', 'Sapporo', 'Sendai', 'Niigata', 'Tokyo', 'Kanazawa', 'Nagano', 'Osaka', 'Fukuyama', 'Kochi', 'Fukuoka', 'Kagoshima', 'Naha']
# city_listより都市名だけを取得したリストを作成…city_namesに重複がないかの確認を軽量化するため
city_list_nameonly = []
for city in city_list:
    city_list_nameonly.append(city['name'])
city_list_nameonly

In [ ]:
from collections import Counter
name_counter = Counter(city_list_nameonly)
print(name_counter)

for name in selected_cities_name:
    count = name_counter.get(name, 0)
    if count == 0:
        print(f'{name}: 見つからない')
    elif count == 1:
        print(f'{name}: 重複なし...OK!')
    else:
        print(f'{name}: 重複あり')

In [ ]:
# 集計したい都市名とAPIから引っ張る緯度経度をくっつけるためにクラスを作る
class City:
    def __init__(self, name, lat, lon, name_ja):
        self.name = name
        self.lat = lat
        self.lon = lon
        self.name_ja = name_ja

    # 確認用
    def city_print(self):
        print(f'name: {self.name}, lat: {self.lat}, lon: {self.lon}, name_ja: {self.name_ja}')

    # dict型にしないとjsonファイルとして出力できないことが判明したため、メソッドを追加
    def todict(self):
        return {
            'name': self.name,
            'lat': self.lat,
            'lon': self.lon,
            'name_ja': self.name_ja,
        }

In [ ]:
selected_cities_list = []

for name in selected_cities_name:
    # urlについて：{name}は都市名、その後ろの,JPは国名指定。これを付けないと海外の同名都市が引っかかってしまう
    url = f'http://api.openweathermap.org/geo/1.0/direct?q={name},JP&limit=1&appid={API_key}'
    jsondata = requests.get(url).json()
    new_city = City(
        name = name,
        lat = jsondata[0]['lat'],
        lon = jsondata[0]['lon'],
        name_ja = jsondata[0]['local_names']['ja']
    )
    selected_cities_list.append(new_city)

In [ ]:
for i in selected_cities_list:
    i.city_print()

In [ ]:
with open("location.json", mode="w", encoding="utf-8") as f:
    json.dump([City.todict(city) for city in selected_cities_list], f, ensure_ascii=False, indent=2)